# Проект: анализ базы данных сервиса для чтения книг по подписке.


## Описание:
В базе данных находятся информация о книгах, издательствах, авторах, а также пользовательские обзоры книг.

### Задачи исследования:
- Посчитать, сколько книг вышло после 1 января 2000 года;
- Для каждой книги посчитать количество обзоров и среднюю оценку;
- Определить издательство, которое выпустило наибольшее число книг толще 50 страниц (не брошюры)
- Определить автора с самой высокой средней оценкой книг, учитывая только книги с 50 и более оценками;
- Посчитать среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

## Подключение к библиотекам и базе данных 

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

## Знакомство с таблицами

In [2]:
# Таблица books
pd.io.sql.read_sql('''
SELECT *
FROM books
LIMIT 5
''', con = engine)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [3]:
# Таблица authors
pd.io.sql.read_sql('''
SELECT * 
FROM authors
LIMIT 5
''', con = engine)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [4]:
# Таблица publishers
pd.io.sql.read_sql('''
SELECT *
FROM publishers
LIMIT 5
''', con = engine)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [5]:
# Таблица ratings
pd.io.sql.read_sql('''
SELECT * 
FROM ratings 
LIMIT 5
''', con = engine)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [6]:
# Таблица reviews
pd.io.sql.read_sql('''
SELECT * 
FROM reviews 
LIMIT 5''', con = engine)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


## Изучение данных, ответы на поставленные задачи

### Сколько книг вышло после 1 января 2000 года?

In [8]:
pd.io.sql.read_sql('''
SELECT COUNT(book_id) 
FROM books
WHERE publication_date>'2000-01-01'
''',con = engine)

,count
0,819


Итак, после 1 января 2000 года вышло 819 книг.

### Для каждой книги посчитать количество обзоров и среднюю оценку

In [9]:
pd.io.sql.read_sql('''
SELECT b.book_id,
b.title,
COUNT(DISTINCT re.text) AS num_of_reviews,
ROUND (AVG(ra.rating), 2) AS average_rating
FROM books as b

LEFT JOIN ratings AS ra ON ra.book_id=b.book_id
LEFT JOIN reviews AS re ON re.book_id=b.book_id
GROUP BY b.book_id
''', con = engine)

,book_id,title,num_of_reviews,average_rating
0,1,'Salem's Lot,2,3.67
1,2,1 000 Places to See Before You Die,1,2.50
2,3,13 Little Blue Envelopes (Little Blue Envelope...,3,4.67
3,4,1491: New Revelations of the Americas Before C...,2,4.50
4,5,1776,4,4.00
...,...,...,...,...
995,996,Wyrd Sisters (Discworld #6; Witches #2),3,3.67
996,997,Xenocide (Ender's Saga #3),3,3.40
997,998,Year of Wonders,4,3.20
998,999,You Suck (A Love Story #2),2,4.50


In [10]:
pd.io.sql.read_sql('''
SELECT b.book_id,
b.title,
COUNT(DISTINCT re.text) AS num_of_reviews,
ROUND (AVG(ra.rating), 2) AS average_rating
FROM books as b

LEFT JOIN ratings AS ra ON ra.book_id=b.book_id
LEFT JOIN reviews AS re ON re.book_id=b.book_id
GROUP BY b.book_id
ORDER BY num_of_reviews DESC

''', con = engine)

,book_id,title,num_of_reviews,average_rating
0,948,Twilight (Twilight #1),7,3.66
1,963,Water for Elephants,6,3.98
2,734,The Glass Castle,6,4.21
3,302,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.41
4,695,The Curious Incident of the Dog in the Night-Time,6,4.08
...,...,...,...,...
995,83,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.67
996,808,The Natural Way to Draw,0,3.00
997,672,The Cat in the Hat and Other Dr. Seuss Favorites,0,5.00
998,221,Essential Tales and Poems,0,4.00


In [11]:
pd.io.sql.read_sql('''
SELECT b.book_id,
b.title,
COUNT(DISTINCT re.text) AS num_of_reviews,
ROUND (AVG(ra.rating), 2) AS average_rating
FROM books as b

LEFT JOIN ratings AS ra ON ra.book_id=b.book_id
LEFT JOIN reviews AS re ON re.book_id=b.book_id
GROUP BY b.book_id
ORDER BY average_rating
''', con = engine)

,book_id,title,num_of_reviews,average_rating
0,303,Harvesting the Heart,2,1.50
1,371,Junky,2,2.00
2,316,His Excellency: George Washington,2,2.00
3,202,Drowning Ruth,3,2.00
4,915,The World Is Flat: A Brief History of the Twen...,3,2.25
...,...,...,...,...
995,421,Marvel 1602,2,5.00
996,418,March,2,5.00
997,169,Crucial Conversations: Tools for Talking When ...,2,5.00
998,901,The Walking Dead Book One (The Walking Dead #...,2,5.00


Число ревью - от 7 до 0, причём нет ревью только у 6 книг, а 7 ревью - только у одной, "Twilight (Twilight #1)"
Рейтинги варьируются  от 1.50 до 5.00, причём рейтинг <=3 получили 102 книги, а >= 4 - 564 книги.

### Издательство, которое выпустило наибольшее число книг толще 50 страниц (не брошюры)

In [12]:
pd.io.sql.read_sql('''
SELECT num_b.publisher,
       num_b.number_of_book

FROM (SELECT COUNT(DISTINCT b.book_id) AS number_of_book,
        pub.publisher     
        FROM books AS b
        LEFT JOIN publishers AS pub ON pub.publisher_id=b.publisher_id
        WHERE b.num_pages>50
        GROUP BY publisher
           ) AS num_b
ORDER BY  number_of_book DESC
LIMIT 1
''', con = engine)

,publisher,number_of_book
0,Penguin Books,42


Наибольшее число книг выпустило издательство Penguin Books	- 42 книги.

### Автор с самой высокой средней оценкой книг, если оценок книги не меньше 50

In [13]:
pd.io.sql.read_sql('''
SELECT a.author,
AVG(ra.rating) AS avg_book_rat
FROM  books AS b
LEFT JOIN authors AS a ON b.author_id=a.author_id
LEFT JOIN ratings AS ra ON ra.book_id=b.book_id
WHERE b.book_id IN (SELECT book_id
                  FROM ratings
                  GROUP BY book_id
                  HAVING COUNT(rating_id) >= 50
                  ) 
GROUP BY a.author_id 
ORDER BY avg_book_rat DESC
LIMIT 1


''', con = engine)

,author,avg_book_rat
0,J.K. Rowling/Mary GrandPré,4.287097


Автор с самой высокой средней оценкой книг - Дж.К.Роулинг. Средняя оценка её книг - 4.29

### Cреднее количество обзоров от пользователей, которые поставили больше 50 оценок

In [14]:
pd.io.sql.read_sql('''
SELECT ROUND(AVG(re_ra.num_rev),0) AS avg_num_of_review

FROM (SELECT username, 
    COUNT(review_id)  AS num_rev
    FROM reviews 
    WHERE username IN 
        (SELECT username 
        FROM ratings 
        GROUP BY username 
        HAVING COUNT(rating_id) > 50) 
    GROUP BY username) AS re_ra
    

''', con = engine)

,avg_num_of_review
0,24.0


Итак, среднее количество обзоров от пользователей, которые поставили больше 50 оценок - 24.

## Вывод:
- После 1 января 2000 года вышло 819 книг.
- Число ревью - от 7 до 0, причём нет ревью только у 6 книг, а 7 ревью - только у одной, "Twilight (Twilight #1)"
- Рейтинги варьируются  от 1.50 до 5.00, причём рейтинг <=3 получили 102 книги, а >= 4 - 564 книги.
- Наибольшее число книг выпустило издательство Penguin Books - 42 книги.
- Автор с самой высокой средней оценкой книг - Дж.К.Роулинг. Средняя оценка её книг - 4.28
- Среднее количество обзоров от пользователей, которые поставили больше 50 оценок - 24.